In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

In [2]:
from datasetgen import *
dataset = get_dataset()

In [3]:
input_texts = []
target_texts = []
input_characters = set(["<end>","<unk>"])
target_characters = set(["<start>","<end>"])
for line in dataset:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '<start> ' + target_text + ' <end>'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text.split():
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text.split():
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt.split()) for txt in input_texts])
max_decoder_seq_length = max([len(txt.split()) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text.split()):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index['<end>']] = 1.
    for t, char in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index['<end>']] = 1.
    decoder_target_data[i, t:, target_token_index['<end>']] = 1.
def encode_input(input_text):  
    encoder_input_data = np.zeros(
        (1, max_encoder_seq_length, num_encoder_tokens),
        dtype='float32')
    for t, char in enumerate(input_text.split()):
        if char in input_token_index.keys():
            encoder_input_data[0, t, input_token_index[char]] = 1.
        else:
            encoder_input_data[0, t, input_token_index["<unk>"]] = 1.
    encoder_input_data[0, t + 1:, input_token_index['<end>']] = 1.
    return encoder_input_data

Number of samples: 411
Number of unique input tokens: 30
Number of unique output tokens: 20
Max sequence length for inputs: 12
Max sequence length for outputs: 17


In [4]:
batch_size = 20  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 128  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(LSTM(latent_dim,return_sequences=True)(encoder_inputs))
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

Train on 328 samples, validate on 83 samples
Epoch 1/100
328/328 [==============================] - 6s 17ms/sample - loss: 1.4543 - accuracy: 0.6542 - val_loss: 0.9368 - val_accuracy: 0.7307
Epoch 2/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.9244 - accuracy: 0.7294 - val_loss: 0.8552 - val_accuracy: 0.7987
Epoch 3/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.8219 - accuracy: 0.7789 - val_loss: 0.6595 - val_accuracy: 0.8420
Epoch 4/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.7081 - accuracy: 0.8052 - val_loss: 0.5796 - val_accuracy: 0.8434
Epoch 5/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.6173 - accuracy: 0.8162 - val_loss: 0.5074 - val_accuracy: 0.8221
Epoch 6/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.5368 - accuracy: 0.8271 - val_loss: 0.4321 - val_accuracy: 0.8455
Epoch 7/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.4869 

Epoch 57/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.0253 - accuracy: 0.9932 - val_loss: 0.0042 - val_accuracy: 1.0000
Epoch 58/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0029 - val_accuracy: 1.0000
Epoch 59/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.0107 - accuracy: 0.9973 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 60/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.0108 - accuracy: 0.9975 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 61/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.0038 - accuracy: 0.9995 - val_loss: 0.0403 - val_accuracy: 0.9865
Epoch 62/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.0132 - accuracy: 0.9966 - val_loss: 0.0085 - val_accuracy: 0.9986
Epoch 63/100
328/328 [==============================] - 0s 1ms/sample - loss: 0.0023 - accuracy: 0.9998 - val_loss: 0.0014 -

In [5]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['<start>']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char+" "

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [6]:
for seq_index in [random.choice([l for l in range(200)]) for r in range(10)]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: multiplication of <unk> and <unk> and store the value in <unk>
Decoded sentence: mul 3 5 equ 11 <end> 
-
Input sentence: subtract <unk> and <unk>
Decoded sentence: sub 2 4 <end> 
-
Input sentence: multiplication of <unk> and <unk> then store the value in <unk>
Decoded sentence: mul 3 5 equ 11 <end> 
-
Input sentence: divide <unk> and <unk> and then store the value in <unk>
Decoded sentence: div 2 4 equ 11 <end> 
-
Input sentence: addition of <unk> and <unk> and then store the value in <unk>
Decoded sentence: add 3 5 equ 12 <end> 
-
Input sentence: substract <unk> and <unk> store in <unk>
Decoded sentence: sub 2 4 equ 7 <end> 
-
Input sentence: initialize <unk> with division of <unk> and <unk>
Decoded sentence: div 6 8 equ 2 <end> 
-
Input sentence: add <unk> and <unk> store in <unk>
Decoded sentence: add 2 4 equ 7 <end> 
-
Input sentence: define  <unk>  <unk>  <unk>  <unk>  <unk> 
Decoded sentence: def 2 def 3 def 4 
-
Input sentence: multiplication of <unk> and <unk>

In [7]:
def translate(line):
    input_seq = encode_input(line)
    decoded_sentence = decode_sequence(input_seq)
    print('Decoded sentence:', decoded_sentence)

In [8]:
translate("add a and b store value in c")

Decoded sentence: add 2 4 equ 7 <end> 


In [9]:
translate("add a and b")

Decoded sentence: add 2 4 <end> 


In [18]:
translate("define integer a string b")

Decoded sentence: def 2 def 3 def 4 


In [13]:
translate("set c to addition of a and b")
translate("set c to multiplication of a and b")
translate("set c to division of a and b")
translate("set c to subtraction of a and b")

Decoded sentence: add 6 8 equ 2 <end> 
Decoded sentence: mul 6 8 equ 2 <end> 
Decoded sentence: div 6 8 equ 2 <end> 
Decoded sentence: sub 6 8 equ 2 <end> 


In [12]:
print(input_characters)

['<end>', '<unk>', 'add', 'addition', 'and', 'declare', 'define', 'difference', 'divide', 'division', 'in', 'initialize', 'integer', 'multiplication', 'multiply', 'of', 'product', 'result', 'set', 'store', 'substract', 'substraction', 'subtract', 'subtraction', 'sum', 'the', 'then', 'to', 'value', 'with']
